## 简单说明

v1版本，实现了extractor和optimizer的css、js组件化，可支持人机交互修改css、js。

In [1]:
import os
import yaml
import re
import gc
import torch
import gradio as gr
from qwen_vl_utils import process_vision_info
from modelscope import AutoTokenizer, AutoModelForCausalLM, Qwen2VLForConditionalGeneration, AutoProcessor
os.environ['GRADIO_ROOT_PATH'] = f"/{os.environ['JUPYTER_NAME']}/proxy/7860"

2025-01-20 17:15:47.628835: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-20 17:15:47.671890: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-20 17:15:48.716170: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
file_prefix = "/mnt/workspace"
config_path = os.path.join(file_prefix, "config.yaml")
with open(config_path, "r", encoding="utf-8") as config_file:
    config = yaml.safe_load(config_file)

In [3]:
def parse_response(pattern, response: str) -> str:
    match = re.search(pattern, response, re.S)
    if match:
        return match.group(1).strip()
    else:
        print("c Error: Response error from LLM.")
        return None

In [4]:
def get_extractor_prompt(file):
    system_content = config["role"]["extractor"]["system_content"]
    user_content = config["role"]["extractor"]["user_content"]
    with open(file, "r", encoding="utf-8") as f:
        source_code = f.read()
    user_content = user_content.format(source_code=source_code)
    return system_content, user_content


def extract_css_and_js(image, file):
    model_name = "Qwen/Qwen2-VL-7B-Instruct"
    extractor = Qwen2VLForConditionalGeneration.from_pretrained(
        model_name, torch_dtype="auto", device_map="auto"
    )
    processor = AutoProcessor.from_pretrained(model_name)
    
    system_content, user_content = get_extractor_prompt(file)
    messages = [
        {"role": "system", "content": system_content},
        {
            "role": "user", 
            "content": [
                {
                    "type": "image",
                    "image": f"file://{image}",
                },
                {"type": "text", "text": user_content},
            ]
        }
    ]
    text = processor.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    image_inputs, video_inputs = process_vision_info(messages)
    inputs = processor(
        text=[text],
        images=image_inputs,
        videos=video_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to("cuda")
    generated_ids = extractor.generate(**inputs, max_new_tokens=8192)
    generated_ids_trimmed = [
        out_ids[len(in_ids): ] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    response = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )[0]
    
    del extractor
    del processor
    gc.collect()
    torch.cuda.empty_cache()
    
    css = parse_response(pattern=r"\[\[## css ##\]\]\n(.*?)\n\[\[## js ##\]\]", response=response)
    js = parse_response(pattern=r"\[\[## js ##\]\]\n(.*?)\n\[\[## completed ##\]\]", response=response)
    return css, js

In [5]:
css, js = extract_css_and_js(os.path.join(file_prefix, "test.png"), os.path.join(file_prefix, "test.html"))
css, js

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

("```css\n* {\n  border: 0;\n  box-sizing: border-box;\n  margin: 0;\n  padding: 0;\n}\n\n:root {\n  --hue: 223;\n  --bg: hsl(var(--hue), 10%, 90%);\n  --fg: hsl(var(--hue), 10%, 10%);\n  --trans-dur: 0.5s;\n  --trans-timing1: cubic-bezier(0.65, 0, 0.35, 1);\n  --trans-timing2: cubic-bezier(0.65, 0, 0.35, 1.5);\n  font-size: calc(56px + (120 - 56) * (100vw - 280px) / (3840 - 280));\n}\n\nbody,\ninput {\n  color: var(--fg);\n  font: 1em/1.5 sans-serif;\n  transition: background-color var(--trans-dur) var(--trans-timing1), color\n      var(--trans-dur) var(--trans-timing1);\n}\n\nbody {\n  background-color: var(--bg);\n  display: flex;\n  height: 100vh;\n}\n\n.switch {\n  display: flex;\n  margin: auto;\n}\n\n.switch__emoji {\n  box-shadow: 0.25em 0.25em 0.125em rgba(0, 0, 0, 0.3);\n  overflow: hidden;\n  pointer-events: none;\n  top: 0.25em;\n  left: 0.25em;\n  width: 1em;\n  height: 1em;\n}\n\n[dir='rtl'] .switch__emoji {\n  right: 0.25em;\n  left: auto;\n}\n\n.switch__emoji,\n.switch_

In [6]:
def get_optimizer_prompt(css, js):
    system_content = config["role"]["optimizer"]["system_content"]
    user_content = config["role"]["optimizer"]["user_content"].format(source_css=css, source_js=js)
    return system_content, user_content


# 处理上传文件并生成代码
def optimize_css_and_js(css, js):
    model_name = "Qwen/Qwen2.5-Coder-7B-Instruct"
    optimizer = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype="auto",
        device_map="auto"
    )
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # 构造 LLM 输入
    system_content, user_content = get_optimizer_prompt(css, js)
    messages = [
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content}
    ]

    # 优化 css、js 代码
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(optimizer.device)

    generated_ids = optimizer.generate(
        **model_inputs,
        max_new_tokens=8192
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]

    # 解析大模型输出，拆分成 js + css
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    del optimizer
    del tokenizer
    gc.collect()
    torch.cuda.empty_cache()
    
    css = parse_response(pattern=r"\[\[## optimized_css ##\]\]\n(.*?)\n\[\[## optimized_js ##\]\]", response=response)
    js = parse_response(pattern=r"\[\[## optimized_js ##\]\]\n(.*?)\n\[\[## completed ##\]\]", response=response)
    
    return css, js

In [7]:
css, js = optimize_css_and_js(css, js)
css, js

2025-01-20 17:18:19,794 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct].
2025-01-20 17:18:19,795 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-7B-Instruct.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

2025-01-20 17:18:38,958 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct].
2025-01-20 17:18:38,958 - modelscope - WARNING - Failed to create symbolic link /mnt/workspace/.cache/modelscope/Qwen/Qwen2.5-Coder-7B-Instruct for /mnt/workspace/.cache/modelscope/Qwen/Qwen2___5-Coder-7B-Instruct.


("```css\n:root {\n  --hue: 223;\n  --bg: hsl(var(--hue), 10%, 90%);\n  --fg: hsl(var(--hue), 10%, 10%);\n  --trans-dur: 0.5s;\n  --trans-timing1: cubic-bezier(0.65, 0, 0.35, 1);\n  --trans-timing2: cubic-bezier(0.65, 0, 0.35, 1.5);\n  font-size: calc(56px + (120 - 56) * (100vw - 280px) / (3840 - 280));\n}\n\n.component {\n  background-color: var(--bg);\n  display: flex;\n  height: 100vh;\n}\n\n.switch {\n  display: flex;\n  margin: auto;\n}\n\n.switch__emoji {\n  box-shadow: 0.25em 0.25em 0.125em rgba(0, 0, 0, 0.3);\n  overflow: hidden;\n  pointer-events: none;\n  top: 0.25em;\n  left: 0.25em;\n  width: 1em;\n  height: 1em;\n}\n\n[dir='rtl'] .switch__emoji {\n  right: 0.25em;\n  left: auto;\n}\n\n.switch__emoji,\n.switch__emoji:before,\n.switch__emoji:after,\n.switch__emoji-eye,\n.switch__emoji-mouth,\n.switch__emoji-face {\n  display: block;\n  position: absolute;\n}\n\n.switch__emoji,\n.switch__emoji:before,\n.switch__emoji:after,\n.switch__emoji-eye,\n.switch__emoji-mouth {\n  bord

In [8]:
# 构建 Gradio 界面
with gr.Blocks() as demo:
    gr.Markdown("<center><h1>组件复制门</h1></center>")
    gr.Markdown("### 输入您想提取的组件截图和源码")
    with gr.Row():
        with gr.Column(scale=1):
            image_input = gr.Image(label="上传图片(png文件)", type="filepath")
        with gr.Column(scale=1):
            file_input = gr.File(label="上传文件", interactive=True, container=True)
            submit_btn = gr.Button("提交")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 提取的 CSS 代码")
            extracted_css = gr.Code(label="CSS 代码", language="css")
        with gr.Column(scale=1):
            gr.Markdown("### 提取的 JS 代码")
            extracted_js = gr.Code(label="JavaScript 代码", language="javascript")
    optimize_btn = gr.Button("优化代码")

    with gr.Row():
        with gr.Column(scale=1):
            gr.Markdown("### 提取的 CSS 代码")
            optimized_css = gr.Code(label="CSS 代码", language="css", interactive=True)
        with gr.Column(scale=1):
            gr.Markdown("### 封装的的 JS 代码")
            optimized_js = gr.Code(label="JavaScript 代码", language="javascript", interactive=True)
    gr.Markdown("<center><h3>封装组件表现效果</h3></center>###")
    ...

    # 绑定交互逻辑
    submit_btn.click(optimize_css_and_js, inputs=[image_input, file_input], outputs=[extracted_css, extracted_js])
    optimize_btn.click(optimize_css_and_js, inputs=[extracted_css, extracted_js], outputs=[optimized_css, optimized_js])
    

# 运行 Gradio 应用
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
